In [1]:
import pynq
from pynq.overlays.base import BaseOverlay
from ctypes import *
import time
import numpy as np

ol = BaseOverlay("base.bit")

In [2]:
print('test print...')

test print...


In [14]:
%%microblaze ol.PMODA

#include <spi.h>
#include <stdint.h>
#include <pyprintf.h>

#define CS_A_PIN	0
#define CS_M_PIN	6
#define MOSI_PIN	1
#define MISO_PIN	2
#define CLK_PIN	3

#define BUFF_SIZE	7


#define ACC_X_L_ADDR     0x28
#define MAG_X_L_ADDR     0x28
#define GYRO_X_L_ADDR    0x18

#define CTRL_REG6_XL_ADDR    0x20

#define X_DIR_SEL       0x01
#define Y_DIR_SEL       0x02
#define Z_DIR_SEL       0x03

#define READ_CFG       1<<7 //bit 7 high
#define WRITE_CFG      0x7F //bit 7 low
#define MS_CFG         0xC0 //11000000

#define STATE_ON       0x01
#define STATE_OFF      0x00

#define CTRL_REG5_XL   0x1F
#define CTRL_REG6_XL   0x20
#define CTRL_REG4      0x1E

#define CTRL_REG1_M    0x20
#define CTRL_REG2_M    0x21
#define CTRL_REG3_M    0x22
#define CTRL_REG4_M    0x23
#define CTRL_REG5_M    0x24

#define CTRL_REG1_G    0x10
#define CTRL_REG2_G    0x11
#define CTRL_REG3_G    0x12

#define GYRO_WHO_AM_I  0x0F

//Parameter dps range : +/- 245dps
#define NAV_GYRO_PAR_G_245DPS 0
//Parameter dps range : +/- 500dps
#define NAV_GYRO_PAR_G_500DPS 1
//Parameter dps range : +/- 2kdps
#define NAV_GYRO_PAR_G_2kDPS 3

int SPI_Write(uint8_t addr, uint8_t val, uint8_t CS)
{
    uint8_t tx_buff[BUFF_SIZE] = {0};
    uint8_t rx_buff[BUFF_SIZE] = {0};
    
    tx_buff[0] = addr & WRITE_CFG;
    tx_buff[1] = val;
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS);
    
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 2);
    
    spi_close(nav);
    
    return 0;
}

uint8_t SPI_Read(uint8_t addr, uint8_t CS)
{
    uint8_t tx_buff[BUFF_SIZE] = {0};
    uint8_t rx_buff[BUFF_SIZE] = {0};
    
    tx_buff[0] = addr | READ_CFG;   
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS);
    
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 2);
    
    spi_close(nav);
    
    return rx_buff[1];
}

int Nav_Acc_Gyro_Pwr(uint8_t state)
{
    uint8_t temp_val = 0x00;
    
    if(state == STATE_ON)
    {  
        //enable acc output
        SPI_Write(CTRL_REG5_XL, 0x38, CS_A_PIN); //00111000
        //pyprintf("CTRL_REG5_XL, %x\r\n", SPI_Read(CTRL_REG5_XL, CS_A_PIN));
        //set odr rate 10Hz of acc
         SPI_Write(CTRL_REG6_XL, 0x20, CS_A_PIN); //00100000
        //pyprintf("CTRL_REG6_XL, %x\r\n", SPI_Read(CTRL_REG6_XL, CS_A_PIN));
        //set odr rate 14.9Hz of gyro
        SPI_Write(CTRL_REG1_G, 0x20, CS_A_PIN); //00100000
        //pyprintf("CTRL_REG1_G, %x\r\n", SPI_Read(CTRL_REG1_G, CS_A_PIN));
        //enable gyro output
        SPI_Write(CTRL_REG4, 0x38, CS_A_PIN);//00111000
        //pyprintf("CTRL_REG4, %x\r\n", SPI_Read(CTRL_REG4, CS_A_PIN));

    }
    else
    {
        pyprintf("--ACC Power OFF--\r\n");
        SPI_Write(CTRL_REG5_XL, 0x00, CS_A_PIN); 
        SPI_Write(CTRL_REG6_XL, 0x00, CS_A_PIN);
        SPI_Write(CTRL_REG4, 0x00, CS_A_PIN);
        SPI_Write(CTRL_REG1_G, 0x00, CS_A_PIN);
    }
    return 0;
}

int Nav_Acc_Pwr(uint8_t state)
{
    uint8_t temp_val = 0x00;
    
    if(state == STATE_ON)
    {
        //enable all three axes
        //pyprintf("--ACC Power ON--\r\n");
        
        SPI_Write(CTRL_REG5_XL, 0x38, CS_A_PIN);
        //pyprintf("CTRL_REG5_XL: %x\r\n", SPI_Read(CTRL_REG5_XL, CS_A_PIN));
        //set 10Hz odr for accelerometer
        SPI_Write(CTRL_REG6_XL, 0x20, CS_A_PIN);
        //pyprintf("CTRL_REG6_XL: %x\r\n", SPI_Read(CTRL_REG6_XL, CS_A_PIN));
    }
    else
    {
        pyprintf("--ACC Power OFF--\r\n");
        
        SPI_Write(CTRL_REG5_XL, 0x00, CS_A_PIN);
        //pyprintf("CTRL_REG5_XL: %x\r\n", SPI_Read(CTRL_REG5_XL, CS_A_PIN));
        SPI_Write(CTRL_REG6_XL, 0x00, CS_A_PIN);
        //pyprintf("CTRL_REG6_XL: %x\r\n", SPI_Read(CTRL_REG6_XL, CS_A_PIN));
        
    }
    return 0;
}

int Nav_Mag_Pwr(uint8_t state)
{
    if(state == STATE_ON)
    {
        //pyprintf("--MAG Power ON--\r\n");
        //set medium performance mode for x and y and 10Hz ODR for MAG, 
        SPI_Write(CTRL_REG1_M, 0x30, CS_M_PIN);
        //pyprintf("CTRL_REG1_M, %x\r\n", SPI_Read(CTRL_REG1_M, CS_M_PIN));
        
        //set scale to +-4Gauss
        SPI_Write(CTRL_REG2_M, 0x00, CS_M_PIN);
        //pyprintf("CTRL_REG2_M, %x\r\n", SPI_Read(CTRL_REG2_M, CS_M_PIN));
        
        //disable I2C and enable SPI read and write operations, 
        //set the operating mode to continuous conversion
        SPI_Write(CTRL_REG3_M, 0x00, CS_M_PIN);
        //pyprintf("CTRL_REG3_M, %x\r\n", SPI_Read(CTRL_REG3_M, CS_M_PIN));
        
        //set medium performance mode for z axis
        SPI_Write(CTRL_REG4_M, 0x04, CS_M_PIN);
        //pyprintf("CTRL_REG4_M, %x\r\n", SPI_Read(CTRL_REG4_M, CS_M_PIN));
        
        //continuous update of output registers
        SPI_Write(CTRL_REG5_M, 0x00, CS_M_PIN);
        //pyprintf("CTRL_REG5_M, %x\r\n", SPI_Read(CTRL_REG5_M, CS_M_PIN));
    }
    else
    {
        //pyprintf("--MAG Power OFF--\r\n");
        //power down the instrument
        SPI_Write(CTRL_REG3_M, 0x03, CS_M_PIN);
    }
}

int Nav_Gyro_GetData(uint8_t axis)
{
    uint8_t tx_buff[7] = {0};
    uint8_t rx_buff[7] = {0};
    int16_t gyro_x = 0;
    int16_t gyro_y = 0;
    int16_t gyro_z = 0;
    
    
    tx_buff[0] = GYRO_X_L_ADDR | READ_CFG;
    
    //see NAV_INST_AG (0b001) used in PmodNav.c as opposed to CS_A_PIN 0
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS_A_PIN);
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 7);
    spi_close(nav);
    
    // Combines the read values for each axis to obtain the 16-bits values
    gyro_x = ((int16_t) rx_buff[2]<<8) | rx_buff[1];
    gyro_y = ((int16_t) rx_buff[4]<<8) | rx_buff[3];
    gyro_z = ((int16_t) rx_buff[6]<<8) | rx_buff[5];
    
    //pyprintf("Nav_Gyro_GetData: x: %d\r\n", gyro_x);
    //pyprintf("Nav_Gyro_GetData: y: %d\r\n", gyro_y);
    //pyprintf("Nav_Gyro_GetData: z: %d\r\n", gyro_z);
    
    if(axis == X_DIR_SEL)
    {
        return gyro_x;
    }
    else if(axis == Y_DIR_SEL)
    {
        return gyro_y;
    }
    else 
    {
        return gyro_z;
    }
    return 0;
}
    
    

int Nav_Acc_GetData(uint8_t axis)
{
    uint8_t tx_buff[7] = {0};
    uint8_t rx_buff[7] = {0};
    int16_t acc_x = 0;
    int16_t acc_y = 0;
    int16_t acc_z = 0;
    
    tx_buff[0] = ACC_X_L_ADDR | READ_CFG;
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS_A_PIN);
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 7);
    spi_close(nav);
    
    acc_x = rx_buff[2]<<8 | rx_buff[1];
    acc_y = rx_buff[4]<<8 | rx_buff[3];
    acc_z = rx_buff[6]<<8 | rx_buff[5];
  
    if(axis == X_DIR_SEL)
    {
        return acc_x;
    }
    else if(axis == Y_DIR_SEL)
    {
        return acc_y;
    }
    else 
    {
        return acc_z;
    }
    return 0;
}

int Nav_Mag_GetData(uint8_t axis)
{
    uint8_t tx_buff[7] = {0};
    uint8_t rx_buff[7] = {0}; 
    int16_t mag_x = 0;
    int16_t mag_y =0;
    int16_t mag_z =0;
    
    tx_buff[0] = MAG_X_L_ADDR | READ_CFG;
    tx_buff[0] = tx_buff[0] | MS_CFG;
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS_M_PIN);
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 7);
    spi_close(nav);
    
    mag_x = rx_buff[2]<<8 | rx_buff[1];
    mag_y = rx_buff[4]<<8 | rx_buff[3];
    mag_z = rx_buff[6]<<8 | rx_buff[5];
    
    if(axis == X_DIR_SEL)
    {
        return mag_x;
    }
    else if(axis == Y_DIR_SEL)
    {
        return mag_y;
    }
    else 
    {
        return mag_z;
    }
    return 0;
}
int simple_func()
{
    
    uint8_t tx_buff[7] = {0};
    uint8_t rx_buff[7] = {0};
    
    int16_t mag_x = 0;
    int16_t mag_y =0;
    int16_t mag_z =0;
    
    tx_buff[0] = GYRO_WHO_AM_I | READ_CFG;
    tx_buff[0] = tx_buff[0] | MS_CFG;
    
    spi nav = spi_open(CLK_PIN, MISO_PIN, MOSI_PIN, CS_A_PIN);
    
    spi_transfer(nav, (char*)tx_buff, (char*)rx_buff, 7);
    
    
    
    //out1 = spi_transfer();
}

float NAV_GetGRangeLSB(uint8_t bRangeG) 
{
    float gRangeLSB;
    pyprintf("Executing NAV_GetGRangeLSB..\n");
    switch (bRangeG) 
    {
        case NAV_GYRO_PAR_G_245DPS:
        gRangeLSB = 0.00875;
        pyprintf("Selecting gRangeLSB: %f\n", gRangeLSB);
            break;
        case NAV_GYRO_PAR_G_500DPS:
        gRangeLSB = 0.0175;
            break;
        case NAV_GYRO_PAR_G_2kDPS:
        gRangeLSB = 0.07;
            break;
        default:
        gRangeLSB = 0.00875;
        break;
   }
    pyprintf("Exiting NAV_GetGRangeLSB...\n");
   return gRangeLSB;
}

float NAV_ConvertReadingToValueDPS(int16_t rawVal)
{
    float gRangeLSB = 0.00875;
    //pyprintf("Calling NAV_GetGRangeLSB...");
    // assuming default is 245 dps, there is a getter and setter for dps
    //gRangeLSB = NAV_GetGRangeLSB(NAV_GYRO_PAR_G_245DPS);
    
    // Convert the gyro value to dps
    float dResult =  rawVal * gRangeLSB;
    //pyprintf("dResult is: %f\n", dResult);
    return dResult;
}


//float NavDemo_ScalarProjection()
/* see use of function in main.c:NAV_DemoRun()*/
//{
    
//}

In [15]:
mag_stop = Nav_Mag_Pwr(0)

acc_gyro_stop = Nav_Acc_Gyro_Pwr(0)

--ACC Power OFF--


# Collecting single samples

In [27]:
acc_gyro_start = Nav_Acc_Gyro_Pwr(1)

mag_start = Nav_Mag_Pwr(1)

gRangeLSB = 0.00875

GX = Nav_Gyro_GetData(0x01)
GY = Nav_Gyro_GetData(0x02)
GZ = Nav_Gyro_GetData(0x03)
print('Gyro Raw: ', GX, ' ', GY, ' ', GZ)

GX_dps = GX * gRangeLSB
GY_dps = GY * gRangeLSB
GZ_dps = GZ * gRangeLSB
print('Gyro DPS: ', GX_dps, ' ', GY_dps, ' ', GZ_dps)
#print("Calc GX_dps\n")
#GX_dps = NAV_ConvertReadingToValueDPS(GX)
#print("Calc GY_dps\n")
#GY_dps = NAV_ConvertReadingToValueDPS(GY)
#print("Calc GZ_dps\n")
#GZ_dps = NAV_ConvertReadingToValueDPS(GZ)
#print(GX_dps, ' ', GY_dps, ' ', GZ_dps)

Gyro Raw:  -28929   -19712   -24321
Gyro DPS:  -253.12875000000003   -172.48000000000002   -212.80875000000003


## Getting raw gyro data and converting to dps

In [28]:
# Turning on gyro
Nav_Acc_Gyro_Pwr(1)

GYRO_RAW = list()
GYRO_DPS = list()
gRangeLSB = 0.00875

T_SAMPLE = 30
FS = 2
N_SAMPLE = T_SAMPLE*FS

for x in range(0, N_SAMPLE):
    acc_gyro_data_x = Nav_Gyro_GetData(0x01)
    acc_gyro_data_y = Nav_Gyro_GetData(0x02)
    acc_gyro_data_z = Nav_Gyro_GetData(0x03)
    
    #GYRO_RAW.append(acc_gyro_data_x)
    #GYRO_RAW.append(acc_gyro_data_y)
    #GYRO_RAW.append(acc_gyro_data_z)
    
    GX_dps = acc_gyro_data_x * gRangeLSB
    GY_dps = acc_gyro_data_y * gRangeLSB
    GZ_dps = acc_gyro_data_z * gRangeLSB
    
    GYRO_DPS.append(GX_dps)
    GYRO_DPS.append(GY_dps)
    GYRO_DPS.append(GZ_dps)
    
    #print("GYRO_RAW:", GYRO_RAW[(x*3) + 0], "\t", GYRO_RAW[(x*3) + 1], "\t", GYRO_RAW[(x*3) + 2], "\r\n")
    print("GYRO_DPS:", GYRO_DPS[(x*3) + 0], "\t", GYRO_DPS[(x*3) + 1], "\t", GYRO_DPS[(x*3) + 2], "\r\n")
    
    time.sleep(1/FS)

GYRO_DPS: -253.12875000000003 	 -181.44000000000003 	 -228.48875 

GYRO_DPS: -248.64875000000004 	 -168.00000000000003 	 -221.76875 

GYRO_DPS: -266.56875 	 -181.44000000000003 	 -215.04875 

GYRO_DPS: -257.60875000000004 	 -181.44000000000003 	 -208.32875 

GYRO_DPS: -253.12875000000003 	 -190.4 	 -203.84875000000002 

GYRO_DPS: -255.36875000000003 	 -172.48000000000002 	 -215.04875 

GYRO_DPS: -250.88875000000002 	 -183.68 	 -215.04875 

GYRO_DPS: -239.68875000000003 	 -161.28000000000003 	 -221.76875 

GYRO_DPS: -266.56875 	 -174.72000000000003 	 -212.80875000000003 

GYRO_DPS: -266.56875 	 -174.72000000000003 	 -219.52875000000003 

GYRO_DPS: -257.60875000000004 	 -192.64000000000001 	 -217.28875000000002 

GYRO_DPS: -277.76875 	 -185.92000000000002 	 -217.28875000000002 

GYRO_DPS: -255.36875000000003 	 -190.4 	 -210.56875000000002 

GYRO_DPS: -271.04875000000004 	 -188.16000000000003 	 -219.52875000000003 

GYRO_DPS: -259.84875000000005 	 -192.64000000000001 	 -210.56875000000002

## Collecting acc and gyro data for ScalarProjection

In [ ]:
def NavDemo_ScalarProjection(ACC, GYRO):
    oM = np.sqrt(ACC[0]*ACC[0] + ACC[1]*ACC[1] + ACC[2]*ACC[2])
    proj = (ACC[0]*GYRO[0] + ACC[1]*GYRO[1] + ACC[2]*GYRO[2]) / oM
    return oM

Nav_Acc_Gyro_Pwr(1)
Nav_Acc_Pwr(1)

ACC = list()
GYRO = list()

gRangeLSB = 0.00875

# Gyro
GX = Nav_Gyro_GetData(0x01)
GY = Nav_Gyro_GetData(0x02)
GZ = Nav_Gyro_GetData(0x03)
#print(GX, ' ', GY, ' ', GZ)

GX_dps = GX * gRangeLSB
GY_dps = GY * gRangeLSB
GZ_dps = GZ * gRangeLSB
print(GX_dps, ' ', GY_dps, ' ', GZ_dps)

GYRO.append(GX_dps)
GYRO.append(GY_dps)
GYRO.append(GZ_dps)

m_GRangeLSB = 0.061
# Accelerometer, NEED RAW DATA to USEFULL DATA CONVERSION
acc_x_flip = -1
acc_y_flip = -1
acc_z_flip = 1

acc_x = acc_x_flip*Nav_Acc_GetData(0x01)*m_GRangeLSB
acc_y = acc_y_flip*Nav_Acc_GetData(0x02)*m_GRangeLSB
acc_z = acc_z_flip*Nav_Acc_GetData(0x03)*m_GRangeLSB

print('acc_x: %6.2f' % acc_x)
print('acc_y: %6.2f' % acc_y)
print('acc_z: %6.2f' % acc_z)

ACC.append(acc_x);
ACC.append(acc_y);
ACC.append(acc_z);
    
dps = NavDemo_ScalarProjection(ACC, GYRO)
if (dps < -4):
    str = "Clockwise"
elif (dps > 4):
    str = "Counter-Clockwise"
else:
    str = "Marginal"

print('    rotating about vertical at %6.2f dps (%s)\n\r' % (dps, str))

## Saving data to a *.txt file

In [11]:
#my_file = open("cal_data_acc.txt", "w")
#for i in range(0,1000):
#    acc_get_data_x = Nav_Acc_GetData(1)
#    acc_get_data_y = Nav_Acc_GetData(2)
#    acc_get_data_z = Nav_Acc_GetData(3)
#    my_file.write('ACC \n')
#    my_file.write(str(acc_get_data_x))
#    my_file.write('\n')
#    my_file.write(str(acc_get_data_y))
#    my_file.write('\n')
#    my_file.write(str(acc_get_data_z))
#    my_file.write("\n \n")
#my_file.close()
my_file = open("cal_data_gyro.txt", "w")
for i in range(0,1000):
    acc_gyro_data_x = Nav_Gyro_GetData(0x01)*gRangeLSB
    acc_gyro_data_y = Nav_Gyro_GetData(0x02)*gRangeLSB
    acc_gyro_data_z = Nav_Gyro_GetData(0x03)*gRangeLSB
    my_file.write('GYRO DPS: \n')
    my_file.write(str(acc_gyro_data_x))
    my_file.write('\n')
    my_file.write(str(acc_gyro_data_y))
    my_file.write('\n')
    my_file.write(str(acc_gyro_data_z))
    my_file.write('\n \n')
my_file.close()    
#my_file = open("cal_data_mag.txt", "w")    
#for i in range(0,1000):
#    mag_data_x = Nav_Mag_GetData(1)
#    mag_data_y= Nav_Mag_GetData(2)
#    mag_data_z= Nav_Mag_GetData(3)
#    my_file.write('MAG \n')
#    my_file.write(str(mag_data_x))
#    my_file.write('\n')
#    my_file.write(str(mag_data_y))
#    my_file.write('\n')
#    my_file.write(str(mag_data_z))
#    my_file.write('\n \n')
#my_file.close()

In [ ]:
if(( 400 <= mag_z < 30000))
    bool right = TRUE 
if((mag_x < && mag_x >=))
    bool back = TRUE
if((mag_x <  && mag_x >=))
    bool front = TRUE
if(mag_x < )
    bool left = TRUE
if(mag_z >= && mag_z < )
   bool forward = TRUE
if(mag_z < )
   bool backward = TRUE

if(forward && right)
   bool correct_backward = TRUE
   bool correct_left = TRUE
if(backward && right)
   bool correct_forward = TRUE
   bool correct_left = TRUE
if(forward && left)
   bool correct_backward = TRUE
   bool correct_right = TRUE
if(backward && left)
   bool correct_forward = TRUE
   bool correct_right = TRUE
